# Comparison of spectrum between eNATL60 outputs and Alitka Satellite 

Requisites :
 - tracks have been computed (results in results_namesat-namemod-region
 - git clone https://github.com/auraoupa/gonzag_cloud
 - git clone https://github.com/brodeau/climporn
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

%matplotlib inline


In [ ]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud/gonzag' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(GONZAG_DIR)
import gonzag as gz

CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


In [2]:
name_mod = 'eNATL60-Region3'
name_ssh_mod = 'sossheig'
name_lsm_mod = 'tmask' 
name_lat_mod = 'lat'
name_lon_mod = 'lon'
l_griddist = False

In [3]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'

## Plot the results

In [32]:
listf=sorted(glob.glob('results_'+name_sat+'-eNATL60-Region3/result_??.nc'))

In [35]:
for f in listf:
    ds=xr.open_dataset(f)
    if ds.sossheig_bl.values.min() < -10:
        print(f)

results_Altika-eNATL60-Region3/result_60.nc
results_Altika-eNATL60-Region3/result_72.nc


### Data

In [6]:
fresults=sorted(glob.glob('results_'+name_sat+'-eNATL60-Region3/result_??.nc'))
dsn=xr.open_mfdataset(fresults,concat_dim='time',combine='nested')

In [7]:
dsn

<xarray.Dataset>
Dimensions:         (time: 5612)
Coordinates:
  * time            (time) float64 1.391e+18 1.391e+18 ... 1.398e+18 1.398e+18
Data variables:
    latitude        (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    sossheig_np     (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    sossheig_bl     (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    distance        (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    time_counter    (time) datetime64[ns] dask.array<chunksize=(75,), meta=np.ndarray>
Attributes:
    about:    Created with Gonzag package => https://github.com/brodeau/gonzag

In [8]:
#clean up some remaining Nans

ds=dsn.where(nmp.isnan(dsn.sla_unfiltered)==0, drop=True)

In [9]:
ds

<xarray.Dataset>
Dimensions:         (time: 5612)
Coordinates:
  * time            (time) float64 1.391e+18 1.391e+18 ... 1.398e+18 1.398e+18
Data variables:
    latitude        (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    sossheig_np     (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    sossheig_bl     (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    distance        (time) float64 dask.array<chunksize=(75,), meta=np.ndarray>
    time_counter    (time) datetime64[ns] dask.array<chunksize=(75,), meta=np.ndarray>
Attributes:
    about:    Created with Gonzag package => https://github.com/brodeau/gonzag

### Maps of the tracks

In [10]:
fig = plt.figure( figsize=(12,9), facecolor='w', edgecolor='k')
ax = plt.subplot(111,projection=ccrs.PlateCarree())
ax.coastlines(resolution="10m")
ax.set_xticks(nmp.arange(-80,30,2), crs=ccrs.PlateCarree())
ax.set_xticklabels(nmp.arange(-80,30,2))
ax.set_yticks(nmp.arange(-50,50,2), crs=ccrs.PlateCarree())
ax.set_yticklabels(nmp.arange(-50,50,2))
ax.xlabels_bottom = False
ax.ylabels_right = False

lon_formatter = cticker.LongitudeFormatter()
lat_formatter = cticker.LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                          alpha=0.5)

idx = ds.longitude
idy = ds.latitude
t = ds.time_counter
cf = ax.scatter(idx, idy, c=t, cmap = 'viridis', marker='.', s=3 )
    
plt.savefig('plots/tracks_'+name_sat+'-'+name_mod+'.png')

### Time series of SSH 

In [21]:
def fullprint(*args, **kwargs):
  from pprint import pprint
  import numpy
  opt = numpy.get_printoptions()
  numpy.set_printoptions(threshold=numpy.inf)
  pprint(*args, **kwargs)
  numpy.set_printoptions(**opt)

In [28]:
print(nmp.mean(ds.sossheig_bl.values))

-4.047551406846379


In [31]:
print(nmp.argmin(ds.sossheig_bl.values))

4520


In [30]:
print(nmp.max(ds.sossheig_bl.values))

0.012062193451934993


In [11]:
del fig, ax

clr_sat = '#AD0000'
clr_mod = '#008ab8'

VT = ds.time_counter ; # time from UNIX Epoch to Matlplotlib friendly...

fig = plt.figure(num = 1, figsize=(20,12), facecolor='w', edgecolor='k')
ax = plt.subplot(111)
plt.hlines(0,nmp.min(ds.time_counter.values),nmp.max(ds.time_counter.values), colors='k',label=None,  zorder=5)
plt.plot(VT, ds.sla_unfiltered-nmp.mean(ds.sla_unfiltered), '.', color=clr_sat, markersize=5, 
         alpha=0.5, label=name_sat, zorder=10)
plt.plot(VT, ds.sossheig_bl-nmp.mean(ds.sossheig_bl), '.', color=clr_mod, markersize=5, 
         alpha=0.5, label=name_mod, zorder=15)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.xticks(rotation='60')
plt.ylabel('SSH [m]')
ax.grid(color='k', linestyle='-', linewidth=0.3)
lgnd = plt.legend(bbox_to_anchor=(0.55, 1.), ncol=1, shadow=True, fancybox=True)

plt.savefig('plots/SSH_'+name_sat+'-'+name_mod+'.png')

### Spectrum

In [12]:
%%time

ISeg_beg, ISeg_end = gz.FindUnbrokenSegments( ds.time, ds.distance, ds.sossheig_bl, \
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
NbSeg, Nsl, IDEDSeg = gz.SegmentSelection(ISeg_beg, ISeg_end, np_valid_seg=70)
XPs, XPm, rdist_sample = gz.Process4FFT( IDEDSeg, ds.distance, ds.sossheig_bl, ds.sla_unfiltered )
Kwn, PwSpc_s, PwSpc_m = gz.ApplyFFT( IDEDSeg, XPs, XPm, rdist_sample )

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod = nmp.mean(PwSpc_m[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s[:,:],axis=0)

# Blabla for the plot:
cinfrm = str(NbSeg)+' segments\n'+str(Nsl)+' points/segment\n'+r'$\Delta$d sat.: '+str(round(rdist_sample,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn, vps_mod, clab1=name_mod+' ("'+name_ssh_mod+'")', clr1=clr_mod, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4)

plt.savefig('plots/spectrum_SSH_'+name_sat+'-'+name_mod+'.png')

NameError: name 'gz' is not defined